In [1]:
import sys
import subprocess

print("Python executable:", sys.executable)

subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "pip"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "openpyxl"])


Python executable: C:\Users\josef\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe


0

In [2]:
import openai
from openai import OpenAI

print(openai.__version__)




2.8.1


In [3]:
#import pandas as pd
#import matplotlib.pyplot as plt
#from vllm import LLM, SamplingParams
import pandas as pd
import matplotlib.pyplot as plt
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="not-needed" 
)

MODEL_NAME = "Qwen2-0.5B-Instruct" 

In [ ]:
import pandas as pd

df = pd.read_excel("./testdata_MIE.xlsx")
df.to_csv("./testdata_MIE.csv", index=False)

df.head()


,PatientAccountID,MPINumber,AssessmentID,Source,CollectedDT,Finding,FindingName,FindingDataType,Value,AdditionalInformation
0,NaN,P-5xxxxxxxx,NaN,Stammdaten,Jul xx 20xx xx:xxAM,Geburtsdatum,NaN,9,xx/19xx,NaN
1,NaN,P-5xxxxxxxx,NaN,Stammdaten,Jun xx 20xx xx:xxPM,Geschlecht,NaN,1,männlich,NaN
2,P-8xxxxxxxxx,P-5xxxxxxxx,NaN,Diagnosen,Jun xx 20xx xx:xxAM,Diagnose,NaN,1,N18.0,Terminale Niereninsuffizienz
3,P-8xxxxxxxxx,P-5xxxxxxxx,NaN,Diagnosen,Jun xx 20xx xx:xxPM,Diagnose,NaN,1,G63.2*,Diabetische Polyneuropathie
4,P-8xxxxxxxxx,P-5xxxxxxxx,NaN,Diagnosen,Jun xx 20xx xx:xxPM,Diagnose,NaN,1,H36.0*,Retinopathia diabetica


In [ ]:
messages = [
    {"role": "user", "content": "Say hello from the Qwen server in one short sentence."}
]

response = client.chat.completions.create(
    model="Qwen/Qwen2-0.5B-Instruct",  
    messages=messages,
    temperature=0.1,
    max_tokens=100,
)

print(response.choices[0].message.content)




Hello from the Qwen server.


In [10]:
from typing import List
import pandas as pd

def build_messages_for_query(df: pd.DataFrame, query: str) -> List[dict]:
    return [
        {
            "role": "system",
            "content": (
                "You are a helpful data analyst. "
                "You receive a pandas DataFrame called df and a question about it. "
                "Write ONLY self-contained Python code using pandas (and matplotlib if needed), "
                "no explanations and no comments. "
                "Always start from the dataframe 'df'. "
                "The dataset is event-based: each patient can have multiple rows. "
                "Patients are identified by the column 'MPINumber'. "
                "Clinical findings such as diagnoses are in the column 'Value' "
                "for rows where 'Finding' == 'Diagnose'. "
                "If the question mentions a diagnosis code like H36 or N18, "
                "you must filter by that code in the 'Value' column and then count "
                "unique patients using the 'MPINumber' column."
            ),
        },
        {
            "role": "user",
            "content": f"""Here are the first rows of the dataframe:
{df.head(5)}

Columns:
{list(df.columns)}

The dataset is event-based: each patient can have multiple rows
with different findings. Patients are identified by MPINumber.
All your code should start from 'df'.
"""
        },
        # Few-shot Beispiel, das genau das richtige Muster zeigt:
        {
            "role": "user",
            "content": "How many patients got diagnosed with N18? Consider that each patient can have multiple findings."
        },
        {
            "role": "assistant",
            "content": (
                "mask_n18 = (df['Finding'] == 'Diagnose') & df['Value'].str.contains('N18', na=False)\n"
                "df_n18 = df.loc[mask_n18]\n"
                "n_patients_n18 = df_n18['MPINumber'].nunique()\n"
                "print('Number of patients diagnosed with N18:', n_patients_n18)"
            )
        },
        # Deine aktuelle Frage
        {
            "role": "user",
            "content": query
        }
    ]

def ask_llm(df: pd.DataFrame, query: str) -> str:
    """
    Schickt die Messages an den Qwen-Server und gibt den Text der Antwort zurück.
    """
    messages = build_messages_for_query(df, query)

    response = client.chat.completions.create(
        model="Qwen/Qwen2-0.5B-Instruct",
        messages=messages,
        temperature=0.1,
        max_tokens=800,
    )

    return response.choices[0].message.content


## RQ - A
How many people got diagnosed with H36 over time?

In [11]:
query = "How many patients got diagnosed with H36? Consider that each patient can have multiple findings."
response_text = ask_llm(df, query)

print("Output:")
print(response_text)



Output:
mask_h36 = (df['Finding'] == 'Diagnose') & df['Value'].str.contains('H36', na=False)
df_h36 = df.loc[mask_h36]
n_patients_h36 = df_h36['MPINumber'].nunique()
print('Number of patients diagnosed with H36:', n_patients_h36)


## RQ - B
How was the age and gender distribution of the disease?

In [ ]:
query = query_template(df, "Given the dataframe above. Create a pie chart for the Gender distribution. " + data_discription_str, tokenizer)

llm_outputs = llm.generate(query, SamplingParams(temperature=0.1, max_tokens=400))
for i, out in enumerate(llm_outputs):
    print('Output:')
    print(out.outputs[0].text)
    print('#########')

## RQ - C
What were the five most common secondary diagnoses?

In [ ]:
query = query_template_chat(df, "Plot the five most common secondary diagnosis", tokenizer)

llm_outputs = llm.generate(query, SamplingParams(temperature=0.1, max_tokens=400))
for i, out in enumerate(llm_outputs):
    print('Output:')
    print(out.outputs[0].text)
    print('#########')

## RQ - D
How many HBA1C values are there for each patient?

In [ ]:
query = query_template_chat(df, "How many the HBA1C values has each diagnosed patient?", tokenizer)

llm_outputs = llm.generate(query, SamplingParams(temperature=0.1, max_tokens=400))
for i, out in enumerate(llm_outputs):
    print('Output:')
    print(out.outputs[0].text)
    print('#########')